Todo 

- [ ] Checar dados faltantes em raca e sexo

In [0]:
import pandas as pd
import numpy as np

# elementos do Bokeh 
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.models import CategoricalColorMapper, Legend, LegendItem
from bokeh.palettes import Spectral9
from bokeh.models.annotations import Legend
from bokeh.io import output_notebook
from bokeh.layouts import column

In [0]:
# Datasets
url_discentes_2019 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/a55aef81-e094-4267-8643-f283524e3dd7/download/discentes-2019.csv"
url_cursos = "http://dados.ufrn.br/dataset/02526b96-cf40-4507-90b0-3afe5ddd53e7/resource/a10bc434-9a2d-491a-ae8c-41cf643c35bc/download/cursos-de-graduacao.csv"

# ColunaS
COLUNAS_ALUNO = ['sexo', 'raca', 'tipo_discente', 'status', 'sigla_nivel_ensino', 
                 'matricula', 'id_curso']
COLUNAS_CURSO = ['nome', 'nivel_ensino', 'area_conhecimento', 'id_curso', 
                 'modalidade_educacao']

discentes = pd.read_csv(url_discentes_2019, sep=';')
alunos_ativos_2019 = (discentes.status == 'ATIVO') & (discentes.sigla_nivel_ensino == 'G')
dados_discentes = discentes.loc[ alunos_ativos_2019 , COLUNAS_ALUNO ]

cursos = pd.read_csv(url_cursos, sep=';')
cursos_graduacao = cursos.loc[ cursos.nivel_ensino == 'GRADUAÇÃO',  COLUNAS_CURSO ]
dist_cursos_2019 = cursos_graduacao.set_index('id_curso')

### Pre processamento

In [3]:
cursos_graduacao[['nome','area_conhecimento']].drop_duplicates()['nome'].value_counts()

PEDAGOGIA                                       2
LETRAS - LÍNGUA PORTUGUESA                      2
PSICOLOGIA                                      2
GEOGRAFIA                                       2
HISTÓRIA                                        1
FISIOTERAPIA                                    1
SERVIÇO SOCIAL                                  1
ENGENHARIA AGRONÔMICA                           1
DESIGN                                          1
ECOLOGIA                                        1
QUÍMICA DO PETRÓLEO                             1
ENGENHARIA DE ALIMENTOS                         1
LETRAS - LÍNGUA ESPANHOLA                       1
EDUCAÇÃO FÍSICA                                 1
TURISMO                                         1
ENGENHARIA DE PRODUÇÃO                          1
ODONTOLOGIA                                     1
TEATRO                                          1
ENGENHARIA DE TELECOMUNICAÇÕES                  1
ENGENHARIA DE AQUICULTURA                       1


In [4]:
cursos_graduacao.head()

,nome,nivel_ensino,area_conhecimento,id_curso,modalidade_educacao
0,ADMINISTRAÇÃO,GRADUAÇÃO,Ciências Sociais Aplicadas,2000002,Presencial
1,ADMINISTRAÇÃO PÚBLICA,GRADUAÇÃO,Ciências Sociais Aplicadas,15315770,A Distância
2,ANÁLISE E DESENVOLVIMENTO DE SISTEMAS,GRADUAÇÃO,Outra,94598200,Presencial
3,ARQUITETURA E URBANISMO,GRADUAÇÃO,Engenharias,2000005,Presencial
4,ARTES VISUAIS,GRADUAÇÃO,"Linguística, Letras e Artes",2000123,Presencial


## Como está hoje?

Vamos começar com a porcentagem de mulheres por curso

In [0]:
# porcentagem de mulheres por curso
cursos_e_alunos = pd.merge(left=cursos_graduacao, right=dados_discentes, on='id_curso')

In [6]:
cursos_e_alunos.columns

Index(['nome', 'nivel_ensino', 'area_conhecimento', 'id_curso',
       'modalidade_educacao', 'sexo', 'raca', 'tipo_discente', 'status',
       'sigla_nivel_ensino', 'matricula'],
      dtype='object')

Usando o pivot_table, podemos usar as matriculas dos discentes para recuperar as informações de sexo e raça. A função de agregação vai ser um count do número de alunos. Quando não há nenhum aluno com alguma das raças ou do sexo, o valor padrão seria NaN. Para isso, passamos `fill_value = 0` na função.

In [0]:
cursos_sexos_count = pd.pivot_table(cursos_e_alunos, values='matricula', index=['id_curso'],                    
            columns=['sexo'], aggfunc=lambda x: len(x), fill_value=0)

In [8]:
cursos_sexos_count.head()

sexo,F,M
id_curso,,
2000002,37,72
2000005,13,8
2000006,23,15
2000009,13,9
2000011,37,56


Passamos os valores encontrados de volta para o dataset principal

In [0]:
dist_cursos_2019['F']= cursos_sexos_count['F']
dist_cursos_2019['M']= cursos_sexos_count['M']

Para os cursos usamos o pivot_table de forma semelhante à usada para contar os sexos. Neste caso, ainda é necessário normalizar os nomes das colunas, retirando os espaços e os caracteres especiais.

In [0]:
cursos_racas_count = pd.pivot_table(cursos_e_alunos, values='matricula', index=['id_curso'],                    
            columns=['raca'], aggfunc=lambda x: len(x), fill_value=0)

cursos_racas_count.columns = cursos_racas_count.columns.str.lower()

names = {
    "amarelo (de origem oriental)": "amarelo",
    "indígeno": "indigeno",
    "não informado": "nao_informado"
}
cursos_racas_count = cursos_racas_count.rename(columns=names)

In [11]:
cursos_racas_count.head()

raca,amarelo,branco,indigeno,negro,nao_informado,pardo
id_curso,,,,,,
2000002,1,54,1,9,1,43
2000005,0,13,0,1,0,7
2000006,0,17,0,4,3,14
2000009,0,10,0,3,1,8
2000011,0,42,0,3,1,47


In [0]:
for raca in cursos_racas_count.columns:
  dist_cursos_2019[raca] = cursos_racas_count[raca]

In [13]:
dist_cursos_2019.fillna(0).loc[(dist_cursos_2019.nome== 'DESIGN'), :]

,nome,nivel_ensino,area_conhecimento,modalidade_educacao,F,M,amarelo,branco,indigeno,negro,nao_informado,pardo
id_curso,,,,,,,,,,,,
6992521,DESIGN,GRADUAÇÃO,"Linguística, Letras e Artes",Presencial,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Por existirem modalidades diferentes, alguns cursos aparecem com apenas um aluno, como é o caso abaixo.

In [14]:
dados_discentes.loc[ dados_discentes.id_curso == 15315770]

,sexo,raca,tipo_discente,status,sigla_nivel_ensino,matricula,id_curso
10311,M,Branco,REGULAR,ATIVO,G,20190000026,15315770.0


Para contornar isso, vamos agrupar todos os cursos, independente da modalidade.

In [15]:
dist_cursos_2019.fillna(0).groupby('nome').get_group('MATEMÁTICA')

,nome,nivel_ensino,area_conhecimento,modalidade_educacao,F,M,amarelo,branco,indigeno,negro,nao_informado,pardo
id_curso,,,,,,,,,,,,
85436489,MATEMÁTICA,GRADUAÇÃO,Ciências Exatas e da Terra,Presencial,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000055,MATEMÁTICA,GRADUAÇÃO,Ciências Exatas e da Terra,Presencial,14.0,22.0,0.0,17.0,0.0,2.0,1.0,16.0
111635066,MATEMÁTICA,GRADUAÇÃO,Ciências Exatas e da Terra,Presencial,25.0,72.0,0.0,34.0,0.0,6.0,4.0,53.0
2000134,MATEMÁTICA,GRADUAÇÃO,Ciências Exatas e da Terra,A Distância,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2000054,MATEMÁTICA,GRADUAÇÃO,Ciências Exatas e da Terra,Presencial,7.0,15.0,0.0,11.0,0.0,2.0,0.0,9.0


In [0]:
colunas = ['F','M','amarelo','branco','indigeno','negro','nao_informado','pardo']
cursos_agrupados = dist_cursos_2019.fillna(0).groupby('nome')

In [17]:
cursos_agrupados.groups.keys()

dict_keys(['ADMINISTRAÇÃO', 'ADMINISTRAÇÃO PÚBLICA', 'ANÁLISE E DESENVOLVIMENTO DE SISTEMAS', 'ARQUITETURA E URBANISMO', 'ARTES VISUAIS', 'BIBLIOTECONOMIA', 'BIOMEDICINA', 'CIÊNCIA DA COMPUTAÇÃO', 'CIÊNCIAS ATUARIAIS', 'CIÊNCIAS BIOLÓGICAS', 'CIÊNCIAS CONTÁBEIS', 'CIÊNCIAS E TECNOLOGIA', 'CIÊNCIAS ECONÔMICAS', 'CIÊNCIAS SOCIAIS', 'COMUNICAÇÃO SOCIAL', 'COMUNICAÇÃO SOCIAL - AUDIOVISUAL', 'COMUNICAÇÃO SOCIAL- PUBLICIDADE E PROPAGANDA', 'DANÇA', 'DESIGN', 'DIREITO', 'ECOLOGIA', 'EDUCAÇÃO FÍSICA', 'ENFERMAGEM', 'ENGENHARIA AGRONÔMICA', 'ENGENHARIA AMBIENTAL', 'ENGENHARIA BIOMÉDICA', 'ENGENHARIA CIVIL', 'ENGENHARIA DE ALIMENTOS', 'ENGENHARIA DE AQUICULTURA', 'ENGENHARIA DE COMPUTAÇÃO', 'ENGENHARIA DE MATERIAIS', 'ENGENHARIA DE PETRÓLEO', 'ENGENHARIA DE PRODUÇÃO', 'ENGENHARIA DE SOFTWARE', 'ENGENHARIA DE TELECOMUNICAÇÕES', 'ENGENHARIA ELÉTRICA', 'ENGENHARIA FLORESTAL', 'ENGENHARIA MECATRÔNICA', 'ENGENHARIA MECÂNICA', 'ENGENHARIA QUÍMICA', 'ENGENHARIA TÊXTIL', 'ESTATÍSTICA', 'FARMÁCIA', 'FILO

In [18]:
cursos_agrupados.get_group('DESIGN')

,nome,nivel_ensino,area_conhecimento,modalidade_educacao,F,M,amarelo,branco,indigeno,negro,nao_informado,pardo
id_curso,,,,,,,,,,,,
6992521,DESIGN,GRADUAÇÃO,"Linguística, Letras e Artes",Presencial,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
dados_2019 = pd.DataFrame(columns = ['nome', 'area_conhecimento'] + colunas)
dados_2019['nome'] = cursos_agrupados.groups.keys()

# TODO: fazer esse loop de forma pythonica
for curso in cursos_agrupados.groups.keys():
  area = cursos_agrupados.get_group(curso).agg(lambda x:x.value_counts().index[0])
  print(area.get('area_conhecimento'))
  dados_2019.loc[dados_2019.nome == curso, 'area_conhecimento'] = area.get('area_conhecimento')
  
  for coluna in colunas:
    grupo = cursos_agrupados.get_group(curso)[coluna]
    dados_2019.loc[dados_2019.nome == curso, coluna] = grupo.sum()
    

Ciências Sociais Aplicadas
Ciências Sociais Aplicadas
Outra
Engenharias
Linguística, Letras e Artes
Ciências Sociais Aplicadas
Ciências Biológicas
Ciências Exatas e da Terra
Ciências Exatas e da Terra
Ciências Biológicas
Ciências Sociais Aplicadas
Ciências Exatas e da Terra
Ciências Sociais Aplicadas
Ciências Humanas
Ciências Humanas
Ciências Humanas
Ciências Humanas
Linguística, Letras e Artes
Linguística, Letras e Artes
Ciências Sociais Aplicadas
Ciências Biológicas
Ciências da Saúde
Ciências da Saúde
Ciências Agrárias
Engenharias
Engenharias
Engenharias
Engenharias
Ciências Biológicas
Engenharias
Engenharias
Engenharias
Engenharias
Ciências Exatas e da Terra
Engenharias
Engenharias
Engenharias
Engenharias
Engenharias
Engenharias
Engenharias
Ciências Exatas e da Terra
Ciências da Saúde
Linguística, Letras e Artes
Ciências da Saúde
Ciências da Saúde
Ciências Exatas e da Terra
Ciências Exatas e da Terra
Ciências Humanas
Ciências Exatas e da Terra
Ciências Sociais Aplicadas
Ciências Hum

In [40]:
dados_2019.head()

,nome,area_conhecimento,F,M,amarelo,branco,indigeno,negro,nao_informado,pardo
0,ADMINISTRAÇÃO,Ciências Sociais Aplicadas,37,72,1,54,1,9,1,43
1,ADMINISTRAÇÃO PÚBLICA,Ciências Sociais Aplicadas,0,1,0,1,0,0,0,0
2,ANÁLISE E DESENVOLVIMENTO DE SISTEMAS,Outra,9,33,0,12,0,2,3,25
3,ARQUITETURA E URBANISMO,Engenharias,13,8,0,13,0,1,0,7
4,ARTES VISUAIS,"Linguística, Letras e Artes",24,20,1,23,0,4,0,16


O próximo passo é calcular as porcentagens para cada curso

In [41]:
print(dados_2019.loc[(dados_2019.F == 0) & (dados_2019.M==0),'nome'])
# Remove cursos que não possuem alunos
remover = dados_2019.loc[(dados_2019.F == 0) & (dados_2019.M==0),'nome']
dados_2019 = dados_2019.drop(remover.index)

dados_2019['mulheres_no_curso(%)'] = dados_2019['F'] / (dados_2019['M'] + dados_2019['F'])

racas = colunas[2:]
total_racas = dados_2019[ racas ].sum(axis='columns')
for raca in racas:
  dados_2019[raca+'_no_curso(%)'] = dados_2019[raca] / total_racas

14                     COMUNICAÇÃO SOCIAL
18                                 DESIGN
27                ENGENHARIA DE ALIMENTOS
50                 GESTÃO DE COOPERATIVAS
51           GESTÃO DE POLÍTICAS PÚBLICAS
55                                 LETRAS
60    LETRAS - LÍNGUA PORTUGUESA E LIBRAS
61       LICENCIATURA EM CIÊNCIAS SOCIAIS
Name: nome, dtype: object


In [42]:
dados_2019.head()

,nome,area_conhecimento,F,M,amarelo,branco,indigeno,negro,nao_informado,pardo,mulheres_no_curso(%),amarelo_no_curso(%),branco_no_curso(%),indigeno_no_curso(%),negro_no_curso(%),nao_informado_no_curso(%),pardo_no_curso(%)
0,ADMINISTRAÇÃO,Ciências Sociais Aplicadas,37,72,1,54,1,9,1,43,0.33945,0.00917431,0.495413,0.00917431,0.0825688,0.00917431,0.394495
1,ADMINISTRAÇÃO PÚBLICA,Ciências Sociais Aplicadas,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
2,ANÁLISE E DESENVOLVIMENTO DE SISTEMAS,Outra,9,33,0,12,0,2,3,25,0.214286,0,0.285714,0,0.047619,0.0714286,0.595238
3,ARQUITETURA E URBANISMO,Engenharias,13,8,0,13,0,1,0,7,0.619048,0,0.619048,0,0.047619,0,0.333333
4,ARTES VISUAIS,"Linguística, Letras e Artes",24,20,1,23,0,4,0,16,0.545455,0.0227273,0.522727,0,0.0909091,0,0.363636


Agora vamos plotar os gráficos usando a biblioteca Bokeh

In [0]:
# Definindo os dados para os graficos
source_negros = ColumnDataSource(data={
    'x'       : dados_2019["negro_no_curso(%)"],
    'y'       : dados_2019["mulheres_no_curso(%)"],
    'curso'   : dados_2019.nome,
    'area'    : dados_2019.area_conhecimento,
})

source_pardos = ColumnDataSource(data={
    'x'       : dados_2019["pardo_no_curso(%)"],
    'y'       : dados_2019["mulheres_no_curso(%)"],
    'curso'   : dados_2019.nome,
    'area'    : dados_2019.area_conhecimento,
})

source_indigenas = ColumnDataSource(data={
    'x'       : dados_2019["indigeno_no_curso(%)"],
    'y'       : dados_2019["mulheres_no_curso(%)"],
    'curso'   : dados_2019.nome,
    'area'    : dados_2019.area_conhecimento,
})

source_brancos = ColumnDataSource(data={
    'x'       : dados_2019["branco_no_curso(%)"],
    'y'       : dados_2019["mulheres_no_curso(%)"],
    'curso'   : dados_2019.nome,
    'area'    : dados_2019.area_conhecimento,
})

source_amarelos = ColumnDataSource(data={
    'x'       : dados_2019["amarelo_no_curso(%)"],
    'y'       : dados_2019["mulheres_no_curso(%)"],
    'curso'   : dados_2019.nome,
    'area'    : dados_2019.area_conhecimento,
})


In [0]:
def create_legend(plot):
  """Given a plot, put the legend outside the graphic
  """
  legend_items = plot.legend[0].items.copy()
  legend = Legend(items=legend_items, location="center")
  plot.legend.visible = False
  plot.add_layout(legend, 'right')

In [0]:
def create_plot(source, x_label, y_label, color_mapper, tooltips=None):
  plot = figure(title='RELAÇÃO ENTRE RAÇA/COR E GÊNERO NOS CURSOS', 
              plot_height = 600, plot_width = 800,
              x_range=(0, 1), x_axis_label=x_label, 
              y_range=(0, 1), y_axis_label=y_label, tooltips=tooltips,
              tools='hover,save')


  # Add the color mapper to the circle glyph
  circ = plot.circle(x='x', y='y', fill_alpha=0.8, source=source,
              color=dict(field='area', transform=color_mapper), legend='area',
             size=10)


  # Coloca a legenda fora do gráfico
  create_legend(plot)
  
  return plot

In [46]:
output_notebook()

# Cria lista de todas as áreas de conhecimento
areas_list = dados_2019.area_conhecimento.unique().tolist()

# Inicializa o color mapper para as legendas
color_mapper = CategoricalColorMapper(factors=areas_list, palette=Spectral9)

# Cria plot comparando alunos negros e mulheres
tool_negros = [
    ("curso", "@curso"),
    ("negros", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_negros = create_plot(source_negros, 'Porcentagem de alunos negros no curso', 
           'Porcentagem de mulheres no curso', color_mapper, tool_negros)

# Cria plot comparando alunos pardos e mulheres
tool_pardos = [
    ("curso", "@curso"),
    ("pardos", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_pardos = create_plot(source_pardos, 'Porcentagem de alunos pardos no curso', 
           'Porcentagem de mulheres no curso', color_mapper, tool_pardos)

# Cria plot comparando alunos indígenas e mulheres
tool_indigenos = [
    ("curso", "@curso"),
    ("indígenos", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_indigenos = create_plot(source_indigenas, 'Porcentagem de alunos indígenas no curso', 
           'Porcentagem de mulheres no curso', color_mapper, tool_indigenos)

# Cria plot comparando alunos brancos e mulheres
tool_brancos = [
    ("curso", "@curso"),
    ("brancos", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_brancos = create_plot(source_brancos, 'Porcentagem de alunos brancos no curso', 
           'Porcentagem de mulheres no curso', color_mapper, tool_brancos)

# Cria plot comparando alunos brancos e mulheres
tool_amarelos = [
    ("curso", "@curso"),
    ("amarelos", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_amarelos = create_plot(source_amarelos, 'Porcentagem de alunos de origem oriental no curso', 
           'Porcentagem de mulheres no curso', color_mapper, tool_amarelos)


layout = column([plot_negros, plot_pardos, plot_indigenos, plot_brancos, plot_amarelos])

## https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html
show(layout)

Loading BokehJS ...

## O que mudou?

## Como estamos em relação ao Brasil?